In [1]:
import torch

from scipy.linalg import sqrtm, inv,pinv

def compute_r_op(X):
    r = torch.einsum('bet, tab -> bea',X,X.T).mean(0)
    r_op = torch.from_numpy(inv(sqrtm(r)))
    return r_op

def prepro_EA_online_ft(X_,dataset):
    X_list = []
    Y_list = []
    sep = 2 if dataset=='Large' else 1
    for X_sub in X_:
        X_list_sub = []
        sqrt_R_s = compute_r_op(torch.cat(X_sub[:sep])).float()
        for X_sess in X_sub:
            X_s = X_sess.float()
            X_list_sub.append(torch.einsum("fe,bet->bft",sqrt_R_s, X_s))
        X_list.append(X_list_sub)
    return X_list

In [2]:
path = '/users/local/simpleconv_datasets/'

In [3]:
for d in ['BNCI','Zhou']:
    X = torch.load(path+d+'/X_s.pt')
    X_EA_s = prepro_EA_online_ft(X,d)
    torch.save(X_EA_s,path+d+'/X_EA_online_ft.pt')

/tmp/ipykernel_1511283/2433316827.py:6: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  r = torch.einsum('bet, tab -> bea',X,X.T).mean(0)
